In [8]:
import numpy as np
import pandas as pd
from assocplots.manhattan import *
import scipy.stats as stats
import sys

In [6]:
dataset = pd.read_csv('../../data/full_dataset/imputed_clean_full_dataset.csv.gz', compression = 'gzip')
dataset = dataset.drop(dataset.columns[0], axis = 1)
labels = dataset['labels']
dataset = dataset.drop('labels', axis = 1)

dataset.head()

,chr10:100003785,chr10:100008785,chr10:100010813,chr10:100013244,chr10:100013438,chr10:100013563,chr10:100015474,chr10:100016738,chr10:100016803,chr10:100017453,...,chrX:99849017,chrX:99849051,chrX:99890204,chrX:99921757,chrX:99931103,chrX:99933584,chrX:99936147,chrX:99941705,chrX:99956157,labels
0,2,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,1,2,2,1
1,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,2,0,2,2,0,1
2,2,0,0,2,0,0,0,0,0,0,...,2,2,0,0,0,0,0,2,2,1
3,0,0,0,0,1,1,0,0,0,0,...,2,2,2,0,0,0,2,2,0,1
4,1,0,0,1,0,0,0,0,0,0,...,2,2,0,0,0,0,1,2,0,1


In [ ]:
chisquared_dict = {}
dataset_len = len(list(dataset))
for index, variant in enumerate(list(dataset)):
    # Making contigency table (GTs_no x 2)  
    table = pd.crosstab(dataset[variant], labels)
    table = table.loc[:, [0, 1]]

    chi2, p, dof, ex = stats.chi2_contingency(table, correction = False)
    chisquared_dict[variant] = [chi2, p]

    sys.stdout.write('\r')
    to_finish = 100 * index/dataset_len
    sys.stdout.write("%.2f" % to_finish + '%')
    #sys.stdout.write('Memory usage:' + str(process.memory_info().rss/1000000) + 'MB')
    sys.stdout.flush()
